In [1]:
"""
clone and install PySDM dependencies in Colab
"""
import os, sys
if 'google.colab' in sys.modules:
    %cd /content
    if not os.path.isdir('PySDM'):
        !git clone --depth 1 https://github.com/atmos-cloud-sim-uj/PySDM.git
        %cd PySDM
    else:
        %cd PySDM
        !git pull
    !pip install -r requirements.txt
    !ldconfig

In [2]:
import os, sys
if 'google.colab' in sys.modules:
    %cd /content/PySDM
else:
    sys.path.insert(0, os.path.join(os.getcwd(), '../..'))

In [3]:
from PySDM_examples.utils.temporary_file import TemporaryFile
from PySDM.physics.constants import si, convert_to
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.settings import Settings
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.simulation import Simulation
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.storage import Storage
from PySDM_examples.Bartman_et_al_2021.progbar_controller import ProgBarController
from PySDM_examples.Bartman_et_al_2021.label import label
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.netcdf_exporter import NetCDFExporter
import PySDM.products as PySDM_products
from PySDM_examples.utils.show_plot import show_plot
from matplotlib import pyplot, rcParams
from scipy.ndimage.filters import uniform_filter1d
from scipy.io.netcdf import netcdf_file
import numpy as np

/home/slayoo/devel/PySDM/PySDM_examples/Bartman_et_al_2021/../../PySDM/backends/__init__.py:29: UserWarning: CUDA library found but cuInit() failed (error code: 999; message: unknown error)
  warnings.warn(


In [4]:
ensemble_size = 3
runs = []
for _ in range(ensemble_size):
    runs.append({'file': TemporaryFile('.nc'), 'settings': {'coalescence_adaptive': True}})
for _ in range(ensemble_size):
    runs.append({'file': TemporaryFile('.nc'), 'settings': {'coalescence_adaptive': False, 'coalescence_substeps':  32}})
for _ in range(ensemble_size):
    runs.append({'file': TemporaryFile('.nc'), 'settings': {'coalescence_adaptive': False, 'coalescence_substeps':   1}})


In [5]:
radius_range = (.5*si.um, 25*si.um)
dt = 32 * si.s


In [ ]:

for i, run in enumerate(runs):
    settings = Settings()

    products = (
        PySDM_products.DynamicWallTime('Coalescence'),
#         PySDM_products.SurfacePrecipitation(),
    )

    settings.n_sd_per_gridbox = 128 # TODO !!! 128 if 'CI' not in os.environ else 32
    settings.grid = (32, 32)
    settings.dt = dt
    settings.condensation_dt_cond_range = (.25*si.s, settings.dt)
    
    settings.mode_1.norm_factor *= 3
    settings.mode_2.norm_factor *= 3
    settings.spectrum_per_mass_of_dry_air.norm_factor *= 3
    settings.simulation_time = 2 * settings.spin_up_time
    settings.output_interval = settings.dt
    settings.condensation_adaptive = True
    settings.condensation_rtol_x = 1e-6
    settings.condensation_rtol_thd = 2e-5/7/7
    settings.condensation_schedule = 'dynamic'
    settings.kappa = .8
    
    for key, value in run['settings'].items():        
        assert hasattr(settings, key)
        setattr(settings, key, value)
    
    storage = Storage()
    simulation = Simulation(settings, storage)
    simulation.reinit(products)

    simulation.run(ProgBarController(f"run {i+1}/{len(runs)}"))
    exporter = NetCDFExporter(storage, settings, simulation, run['file'].absolute_path)
    exporter.run(ProgBarController('netCDF'))

FloatProgress(value=0.0, description='run 1/9', max=1.0)

In [ ]:
rcParams["figure.figsize"] = (15, 5)

timeaxis = lambda t: t / 3600 - 1

dt_day = dt
convert_to(dt_day, si.day)

window=5
colors=['red', 'green', 'blue']

for i, run in enumerate(runs):
    nc = netcdf_file(run['file'].absolute_path, mode='r', mmap=False)
    wall_time = np.mean(nc.variables['Coalescence_wall_time'][:-10])  # TODO !!!
    wall_time = int(10000 * wall_time) / 10000
    filtered_cumsum = uniform_filter1d(dt_day * np.cumsum(nc.variables['surf_precip'][:]), size=window)
    pyplot.plot(
        timeaxis(nc.variables['T'][:]),
        filtered_cumsum,
        label=f"{label(run['settings'])} (time/step: {wall_time:.4f}s)",
        color=colors[i//ensemble_size],
        lw=.75
    )
    if i % ensemble_size == 0:
        mean = np.copy(filtered_cumsum)
    else:
        mean += filtered_cumsum
    if (i+1) % ensemble_size == 0:
        pyplot.plot(
            timeaxis(nc.variables['T'][:]),
            mean / ensemble_size,
            lw=3,
            color=colors[i // ensemble_size]
        )
pyplot.xlim(timeaxis(1.25 * settings.spin_up_time), timeaxis(nc.variables['T'][-1]))
pyplot.legend()
pyplot.ylabel('accumulated rainfall [mm] ' + f'({window}-point moving average)')
pyplot.xlabel('time after spinup [h]')
show_plot()


for i, run in enumerate(runs):
    if not run['settings']['coalescence_adaptive']:
        continue
    nc = netcdf_file(run['file'].absolute_path, mode='r', mmap=False)
    data = nc.variables['dt_coal_min'][:]
    pyplot.plot(timeaxis(nc.variables['T'][:]), np.amax(np.amax(data, axis=-1), axis=-1))
    pyplot.plot(timeaxis(nc.variables['T'][:]), np.amin(np.amin(data, axis=-1), axis=-1))
show_plot()

In [ ]:
# TODO #432: initialisation
# TODO #432: rng_reuse
# TODO #432: seed, spread
# TODO #432: sd_num ?
# TODO #432: plot deficit
# TODO #432: dt histogram